In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

#### PART 1: DYNAMIC WEIGHT ALLOCATION RULES

In [ ]:
def assign_weights(rule_name, returns_df):

    if rule_name == 'equal':
        # Rule 1: Equal-Weight Rule
        num_assets = len(returns_df.columns)
        return pd.Series(1 / num_assets, index=returns_df.columns)

    elif rule_name == 'risk_parity':
        # Rule 2: Risk-Parity Rule
        # Lower volatility assets get higher weight.
        volatilities = returns_df.std()
        # Add a small epsilon to avoid division by zero for assets with zero volatility
        inverse_volatilities = 1 / (volatilities + 1e-6)
        return inverse_volatilities / inverse_volatilities.sum()

    elif rule_name == 'sharpe_max':
        # Rule 3: Sharpe-Maximization Rule
        # Assets with better risk-adjusted returns get more weight.
        # Assuming risk-free rate is 0 for simplicity.
        mean_returns = returns_df.mean()
        volatilities = returns_df.std()
        sharpe_ratios = mean_returns / (volatilities + 1e-6)
        # Set negative Sharpe ratios to 0 so they get no weight
        positive_sharpe = sharpe_ratios.clip(lower=0)
        return positive_sharpe / positive_sharpe.sum()

    elif rule_name == 'momentum':
        # Rule 4: Momentum Rule
        # Assets that have been going up recently get more weight.
        # We'll measure momentum as the cumulative return over the period.
        total_returns = (1 + returns_df).prod() - 1
        # Set negative momentum to 0 so losing assets get no weight
        positive_momentum = total_returns.clip(lower=0)
        return positive_momentum / positive_momentum.sum()

    else:
        raise ValueError(f"Unknown rule: {rule_name}")

#### PART 2: STRESS TEST SIMULATION

In [ ]:
def run_stress_test():

    # --- 1. Define the Stress Test Data (from your slide) ---
    stress_test_data = {
        "Bull Market": pd.DataFrame({
            "BTC": [0.05, 0.04, 0.03, 0.06],
            "ETH": [0.04, 0.03, 0.02, 0.05],
            "LTC": [0.03, 0.02, 0.01, 0.04]
        }),
        "Bear Market": pd.DataFrame({
            "BTC": [-0.05, -0.04, -0.03, -0.06],
            "ETH": [-0.04, -0.03, -0.02, -0.05],
            "LTC": [-0.03, -0.02, -0.01, -0.04]
        }),
        "Volatile Market": pd.DataFrame({
            "BTC": [0.10, -0.10, 0.12, -0.08],
            "ETH": [0.08, -0.07, 0.09, -0.06],
            "LTC": [0.07, -0.05, 0.06, -0.04]
        })
    }

    rules_to_test = ['equal', 'risk_parity', 'sharpe_max', 'momentum']

    final_results = []

    # --- 2. Loop through each scenario and apply each rule ---
    for scenario_name, returns_data in stress_test_data.items():
        print("\n" + "="*50)
        print(f"STRESS TEST SCENARIO: {scenario_name}")
        print("="*50)

        for rule in rules_to_test:
            # a. Dynamically assign weights based on the data in this scenario
            weights = assign_weights(rule, returns_data)

            # b. Calculate the portfolio's performance under this rule
            portfolio_daily_returns = (returns_data * weights).sum(axis=1)

            # c. Calculate final metrics
            total_return = (1 + portfolio_daily_returns).prod() - 1
            volatility = portfolio_daily_returns.std()

            # d. Store the results
            final_results.append({
                'Scenario': scenario_name,
                'Rule': rule,
                'Total_Return (%)': round(total_return * 100, 2),
                'Volatility (Risk)': round(volatility, 4)
            })

            print(f"--- Rule: {rule} ---")
            print("Calculated Weights:")
            print(weights.round(2))
            print(f"Resulting Total Return: {total_return:.2%}\n")

    # --- 3. Present the Final Summary ---
    summary_df = pd.DataFrame(final_results)

    print("\n" + "="*80)
    print("FINAL STRESS TEST SUMMARY")
    print("="*80)
    print(summary_df)

    print("\n--- Key Insights ---")
    print("1. Bull Market: All strategies are profitable, but 'Momentum' and 'Sharpe-Max' excel by concentrating on the best performers.")
    print("2. Bear Market: All strategies lose money, but 'Risk-Parity' is the most defensive, losing the least by favoring the most stable asset.")
    print("3. Volatile Market: 'Risk-Parity' often performs best here as it avoids the assets with the wildest downswings.")

In [ ]:
if __name__ == "__main__":
    run_stress_test()


STRESS TEST SCENARIO: Bull Market
--- Rule: equal ---
Calculated Weights:
BTC    0.33
ETH    0.33
LTC    0.33
dtype: float64
Resulting Total Return: 14.73%

--- Rule: risk_parity ---
Calculated Weights:
BTC    0.33
ETH    0.33
LTC    0.33
dtype: float64
Resulting Total Return: 14.73%

--- Rule: sharpe_max ---
Calculated Weights:
BTC    0.43
ETH    0.33
LTC    0.24
dtype: float64
Resulting Total Return: 15.57%

--- Rule: momentum ---
Calculated Weights:
BTC    0.43
ETH    0.33
LTC    0.23
dtype: float64
Resulting Total Return: 15.62%


STRESS TEST SCENARIO: Bear Market
--- Rule: equal ---
Calculated Weights:
BTC    0.33
ETH    0.33
LTC    0.33
dtype: float64
Resulting Total Return: -13.31%

--- Rule: risk_parity ---
Calculated Weights:
BTC    0.33
ETH    0.33
LTC    0.33
dtype: float64
Resulting Total Return: -13.31%

--- Rule: sharpe_max ---
Calculated Weights:
BTC   NaN
ETH   NaN
LTC   NaN
dtype: float64
Resulting Total Return: 0.00%

--- Rule: momentum ---
Calculated Weights:
BTC   